# **<center>Using CNN to stopping the car when it comes off the road<center>**
    
In this notebook, you can download images obtained with the donkey car. You can use them to launch a training in order to get a model for the donkey car.
Several options are available :

- The number of image to learn from
- The possibility to invert them to also learn turning right
    
    

In [ ]:
import json
import shutil
import random
import pathlib
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import IPython.display as display

import mlflow
import mlflow.tensorflow
import mlflow.keras

import tensorflow as tf

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import MSLE, MAE, MSE
from tensorflow.keras.losses import MSE, MSLE
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from sklearn.metrics import precision_recall_curve

from xebikart.images import transformer as T
import xebikart.dataset as dataset

from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available()

Eager Execution allows to evaluate operations immediately without building graphs
note : Only needed when not using TF 2.0

In [ ]:
tf.enable_eager_execution()

In [ ]:
# parameters
# dataset parameters
tubes_root_folder = 'file:/workspace/xebikart-ml-tubes'
tubes_folders_exit = ["tub.v7.01", "tub.v7.02"]
tubes_folders_road = ["tub.v5.01", "tub.v5.02"]#, "tub.v5.03", "tub.v5.04"]

test_size=0.2

# training parameters
batch_size = 32
shuffle_size = 200
n_epochs = 10
learning_rate = 1e-3

# Load data

Download tubes from : https://github.com/xebia-france/xebikart-ml-tubes

In [ ]:
%%capture
road_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders_road, tubes_extension=".tar.gz")
road_tubes_df['label'] = 0
exit_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders_exit, tubes_extension=".tar.gz")
exit_tubes_df['label'] = 1
road_tubes_df = road_tubes_df.rename(columns={"cam/image_array": "images_path"})
exit_tubes_df = exit_tubes_df.rename(columns={"cam/image_array": "images_path"})
tubes_df = pd.concat([road_tubes_df, exit_tubes_df]).reset_index(drop=True)
tubes_df = tubes_df.drop(["user/angle", "user/throttle", "user/mode", "timestamp"], axis=1)

In [ ]:
tubes_df.head()

#### **- Display some examples**

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,5), constrained_layout=True)
fig.suptitle("label", fontsize=20)

for n, sample in tubes_df.sample(3).reset_index().iterrows():
    random_image_path = sample["images_path"]
    label = sample["label"]
    image = mpimg.imread(random_image_path) 
    axs[n].set_title(label)
    axs[n].imshow(image)
    axs[n].get_xaxis().set_visible(False)
    axs[n].get_yaxis().set_visible(False)

#### **- Display some sample distribution**

In [ ]:
labels = 'on the road', 'Out of the road'
sizes = [len(tubes_df[tubes_df.label == 0]), len(tubes_df[tubes_df.label == 1])]
explode = (0, 0.1)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

# Preprocessing Images

In [ ]:
crop_fn = T.generate_crop_fn(left_margin=0, width=160, height_margin=80, height=40)

def load_augmentation_preprocess(image_path):
    tf_image = T.read_image(image_path)
    tf_image = T.normalize(tf_image)
    tf_image = tf.image.rgb_to_grayscale(tf_image)
    tf_image = crop_fn(tf_image)
    return tf_image

In [ ]:
random_image_path = tubes_df.sample()["images_path"].values[0]

tf_image_original   = T.read_image(random_image_path)
tf_image_normalized = T.normalize(tf_image_original)/255.0
tf_image_cropped    = crop_fn(tf_image_normalized)
tf_image_grayscaled = tf.image.rgb_to_grayscale(tf_image_cropped)

fig, axs = plt.subplots(1, 3, figsize=(15,15), constrained_layout=True)
axs[0].set_title("Original")
axs[0].imshow(tf_image_original)
axs[1].set_title("Cropping & normalized")
axs[1].imshow(tf_image_cropped)
axs[2].set_title("Preprocessed")
axs[2].imshow(tf_image_grayscaled[:,:,0],cmap='gray')
plt.show()

# Build a dataset of images

#### **- Split data into test/train datasets**

Note : We only use angle as label

In [ ]:
images_path = tubes_df["images_path"].tolist()
label = tubes_df["label"].tolist()

In [ ]:
train_images_path, test_images_path, train_metas, test_metas = train_test_split(images_path, label, test_size=test_size)
print('Train set :', len(train_images_path), 'images')
print('Test set :', len(test_images_path), 'images')

#### **- Create tensor for train and test datasets**

In [ ]:
def input_fn(filepath, label, BATCH_SIZE = 32, SHUFFLE_SIZE = 200, NUM_EPOCHS = 50):
    ds_x = tf.data.Dataset.from_tensor_slices(filepath)
    ds_x = ds_x.map(load_augmentation_preprocess)
    ds_y = tf.data.Dataset.from_tensor_slices(label)
    ds_x_y = tf.data.Dataset.zip((ds_x, ds_y)).shuffle(SHUFFLE_SIZE).repeat(NUM_EPOCHS).batch(BATCH_SIZE).prefetch(1)
    
    return ds_x_y

In [ ]:
ds_train = input_fn(train_images_path, train_metas)
ds_test = input_fn(test_images_path, test_metas, NUM_EPOCHS=1)

# Build Model

In [ ]:
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(40, 160,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate, decay=learning_rate/n_epochs), loss="binary_crossentropy")

In [ ]:
mlflow.set_experiment("detect_road_exit")

with mlflow.start_run():
    mlflow.log_params({
        "images_exit": str(tubes_folders_exit),
        "images_road": str(tubes_folders_road),
        "nb_images": len(train_images_path),
        "epochs": n_epochs,
        "batch_size": batch_size,
        "learning_rate": learning_rate
    })
    mlflow.tensorflow.autolog()
    history = model.fit(x=ds_train,
                    steps_per_epoch=len(train_metas)//batch_size,
                    epochs=n_epochs,
                    verbose=1,
                    validation_data=ds_test,
                    validation_steps=len(test_metas)//batch_size)

# Plot loss

In [ ]:
model.evaluate(ds_test)

In [ ]:
def plot_results(history):
    hist_df = pd.DataFrame(history.history)
    hist_df.columns=['loss','val_loss']
    hist_df.index = np.arange(1, len(hist_df)+1)
    fig = plt.figure(figsize=(10,5))
    plt.plot(hist_df.val_loss, lw=3, label='Validation Loss')
    plt.plot(hist_df.loss, lw=3, label='Training Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.legend(loc=0)

    plt.show()

In [ ]:
plot_results(history)

# Evaluation & Visualisation

### - Prediction

In [ ]:
test_pred = model.predict(ds_test, use_multiprocessing=True, workers=12, steps = None)#len(test_metas)/batch_size)

In [ ]:
df = pd.DataFrame(test_pred, columns = ['label'])
df.label.hist()

### - Visualisation predictions

In [ ]:
nb_random = np.random.randint(len(test_metas))

random_image_path_0 = [(test_images_path[nb_random])]
random_image_label_0 = [test_metas[nb_random]]

ds_visu_test_0 = input_fn(random_image_path_0, random_image_label_0)

nb_random = np.random.randint(len(test_metas))

random_image_path_1 = [(test_images_path[nb_random])]
random_image_label_1 = [test_metas[nb_random]]

ds_visu_test_1 = input_fn(random_image_path_1, random_image_label_1)

fig, axs = plt.subplots(1, 2, figsize=(15,5), constrained_layout=True)
axs[0].set_title("Value ({}) : Predict ({})".format(random_image_label_0, model.predict(ds_visu_test_0)[0]))
axs[0].imshow(T.read_image(random_image_path_0[0]))
axs[1].set_title("Value ({}) : Predict ({})".format(random_image_label_1, model.predict(ds_visu_test_1)[0]))
axs[1].imshow(T.read_image(random_image_path_1[0]))

plt.show()

### - Interpretation results

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true=test_metas, probas_pred=test_pred)

In [ ]:
plt.plot(thresholds, precision[:-1], '--', label='precision')
plt.plot(thresholds, recall[:-1], '--', label='recall')
plt.xlabel('threshold')
plt.legend()

plt.show()